In [1]:
import numpy as np
import os
import pickle
import copy
import edt
import matplotlib.pyplot as plt
import time
import cv2
import pandas as pd
from sklearn.metrics.cluster import adjusted_rand_score
from skimage.metrics import adapted_rand_error

import torch
from torch import from_numpy as from_numpy
from torchsummary import summary

from func.run_pipeline_super_vox import segment_super_vox_3_channel, semantic_segment_crop_and_cat_3_channel_output, img_3d_erosion_or_expansion, \
generate_super_vox_by_watershed, get_outlayer_of_a_3d_shape, get_crop_by_pixel_val, Cluster_Super_Vox, assign_boudary_voxels_to_cells_with_watershed, \
delete_too_small_cluster, reassign
from func.run_pipeline import segment, assign_boudary_voxels_to_cells, dbscan_of_seg, semantic_segment_crop_and_cat
from func.cal_accuracy import IOU_and_Dice_Accuracy, VOI
from func.network import VoxResNet, CellSegNet_basic_lite
from func.unet_3d_basic import UNet3D_basic
from func.ultis import save_obj, load_obj

from func.graph_learning import SuperVoxToNxGraph, VoxelGraphDataset

Using backend: pytorch


In [2]:
HMS_data_dict = load_obj("dataset_info/HMS_dataset_info")
HMS_data_dict_train = HMS_data_dict["train"]

In [3]:
img_ws_predictions = np.load("img_ws_predictions.npy", allow_pickle=True)
img_ws_predictions = img_ws_predictions[()]

In [4]:
graphs_list = []
for img_name in HMS_data_dict_train.keys():
    if img_name == ".DS_Store":
        continue
    print(img_name)
    raw_img_path = HMS_data_dict_train[img_name]["raw"]
    print(HMS_data_dict_train[img_name]["raw"])
    hand_seg_path = f"data/CellSeg_dataset/HMS_processed/segmentation/{img_name}/{img_name}_ins.npy"

    # raw_img = np.load(raw_img_path)
    hand_seg = np.load(hand_seg_path)

    model_seg = img_ws_predictions[img_name]

    super_vox_to_graph = SuperVoxToNxGraph()
    graph = super_vox_to_graph.get_nx_graph_from_ws_with_gt(model_seg, hand_seg)

    graphs_list.append(graph)

95
data/CellSeg_dataset/HMS_processed/raw/95.npy
getting neighbor pairs
adding ground truth
adding neighbor ids
calculate edges
build networkx graph
50
data/CellSeg_dataset/HMS_processed/raw/50.npy
getting neighbor pairs
adding ground truth
adding neighbor ids
calculate edges
build networkx graph
150
data/CellSeg_dataset/HMS_processed/raw/150.npy
getting neighbor pairs
adding ground truth
adding neighbor ids
calculate edges
build networkx graph
195
data/CellSeg_dataset/HMS_processed/raw/195.npy
getting neighbor pairs
adding ground truth
adding neighbor ids
calculate edges
build networkx graph
105
data/CellSeg_dataset/HMS_processed/raw/105.npy
getting neighbor pairs
adding ground truth
adding neighbor ids
calculate edges
build networkx graph
60
data/CellSeg_dataset/HMS_processed/raw/60.npy
getting neighbor pairs
adding ground truth
adding neighbor ids
calculate edges
build networkx graph
160
data/CellSeg_dataset/HMS_processed/raw/160.npy
getting neighbor pairs
adding ground truth
adding

In [5]:
save_obj(graphs_list, "graphs_dataset_train")